In [1]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
lbl = LabelEncoder()
mlb = MultiLabelBinarizer()
import numpy as np

In [2]:
df = pd.read_pickle("./dataset.pkl")

In [23]:
#if you want different combination of train and test set execute ml first before this 
df = pd.read_pickle("./dataset.pkl")
x = df.iloc[:,:483].values
y = df.iloc[:,483:].values


In [24]:
import keras
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense,CuDNNLSTM,CuDNNGRU,Flatten,Conv1D,Dropout,Activation,LeakyReLU,Input,MaxPooling1D
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold

In [25]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

In [26]:
x_train.shape

(44232, 483)

In [27]:
y_train.shape

(44232, 20)

In [28]:
x_train = np.reshape(x_train,(x_train.shape[0],1,x_train.shape[1]))
y_train = np.reshape(y_train,(y_train.shape[0],1,y_train.shape[1]))

In [45]:
class_weights = {0:3032/337481,1:8924/337481,2:35608/337481,3:19366/337481,
                 4:16271/337481,5:20008/337481,6:43243/337481,
                 7:25281/337481,8:21200/337481,9:21782/337481,10:166/337481,
                 11:8316/337481,12:10053/337481,13:3205/337481,
                 14:5492/337481,15:19454/337481,16:1757/337481,17:23352/337481,18:16621/337481,19:34350/337481}

In [30]:
model = Sequential()
model.add(Conv1D(filters=64,kernel_size=1,input_shape=(None,x_train.shape[2])))
model.add(LeakyReLU(alpha=0.4))
model.add(CuDNNLSTM(483,return_sequences=True))
model.add(LeakyReLU(alpha=0.4))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64,kernel_size=1))
model.add(CuDNNGRU(483,return_sequences=True))
model.add(LeakyReLU(alpha=0.4))
model.add(Dense(161))
model.add(Dense(81))
model.add(Dense(y_train.shape[2],activation='sigmoid'))

In [31]:
model.compile(metrics=['binary_accuracy','accuracy'],loss='binary_crossentropy',optimizer = 'adam')

In [46]:
model.fit(x_train,y_train,epochs=10,batch_size=64,validation_split=0.25)

ValueError: `class_weight` not supported for 3+ dimensional targets.

In [33]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, None, 64)          30976     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, None, 64)          0         
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, None, 483)         1060668   
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, None, 483)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 483)         0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 64)          30976     
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (None, None, 483)        

In [34]:
x_test = np.reshape(x_test,(x_test.shape[0],1,x_test.shape[1]))

In [42]:
y_test = np.reshape(y_test,(y_test.shape[0],1,y_test.shape[1]))

In [38]:
y_predtest = model.predict(x_test)

In [36]:
y_predtrain = model.predict(x_train)

In [41]:
y_predtrain.shape

(44232, 1, 20)

In [39]:
y_pred = np.argmax(y_predtest,axis=1)

In [40]:
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
#cm = confusion_matrix(y_test,y_pred) 
#Classification metrics can't handle a mix of multilabel-indicator and unknown targets
print(jaccard_score(y_test,y_pred,average='micro'))
print(jaccard_score(y_test,y_pred,average='macro'))
print(roc_auc_score(y_test,y_pred))
print(hamming_loss(y_test,y_pred))

0.0
0.0
0.5
0.2852109332609875


In [43]:
y_predtest_new = np.subtract(y_test,y_predtest)
y_predtrain_new = np.subtract(y_train,y_predtrain)

In [44]:
model.compile(metrics=['binary_accuracy','accuracy'],loss='binary_crossentropy',optimizer = 'adam')
model.fit(x_train,y_predtrain_new,epochs=10,batch_size=64,validation_split=0.25)

Train on 33174 samples, validate on 11058 samples
Epoch 1/10
33174/33174 [==============================] - 8s 248us/step - loss: -471.4342 - binary_accuracy: 1.5072e-06 - accuracy: 1.5072e-06 - val_loss: -2121.0246 - val_binary_accuracy: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
33174/33174 [==============================] - 7s 221us/step - loss: -9198.6341 - binary_accuracy: 1.5072e-06 - accuracy: 1.5072e-06 - val_loss: -18527.1197 - val_binary_accuracy: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
33174/33174 [==============================] - 7s 213us/step - loss: -40667.3166 - binary_accuracy: 1.5072e-06 - accuracy: 1.5072e-06 - val_loss: -62022.1860 - val_binary_accuracy: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
33174/33174 [==============================] - 7s 210us/step - loss: -104645.2607 - binary_accuracy: 1.5072e-06 - accuracy: 1.5072e-06 - val_loss: -137840.3222 - val_binary_accuracy: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
33174/33174 [========